## This notebook is used to generate the finalized version of the classifier, to simply feature transformation into the final form, and to test that the results are the same

Most of the code comes from operational_classifier.

In [46]:
import pandas as pd
import numpy as np
import pickle
from joblib import *
#reload(sys)
#sys.setdefaultencoding("utf-8")

#Loading raw data
df = pd.read_csv("../Data/labeled_data.csv")
tweets = df.tweet


In [47]:
len(tweets)

24783

## Feature generation

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
import string
import re

stopwords=stopwords = nltk.corpus.stopwords.words("english")

other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)

stemmer = PorterStemmer()


def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    #parsed_text = parsed_text.code("utf-8", errors='ignore')
    return parsed_text

def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    tweet = " ".join(re.split("[^a-zA-Z]*", tweet.lower())).strip()
    #tokens = re.split("[^a-zA-Z]*", tweet.lower())
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens

def basic_tokenize(tweet):
    """Same as tokenize but without the stemming"""
    tweet = " ".join(re.split("[^a-zA-Z.,!?]*", tweet.lower())).strip()
    return tweet.split()

vectorizer = TfidfVectorizer(
    #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    tokenizer=tokenize,
    preprocessor=preprocess,
    ngram_range=(1, 3),
    stop_words=stopwords, #We do better when we keep stopwords
    use_idf=True,
    smooth_idf=False,
    norm=None, #Applies l2 norm smoothing
    decode_error='replace',
    max_features=10000,
    min_df=5,
    max_df=0.501
    )

In [49]:
#Construct tfidf matrix and get relevant scores
tfidf = vectorizer.fit_transform(tweets).toarray()
vocab = {v:i for i, v in enumerate(vectorizer.get_feature_names())}
idf_vals = vectorizer.idf_
idf_dict = {i:idf_vals[i] for i in vocab.values()} #keys are indices; values are IDF scores

C:\Users\obloc_000\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [69]:
vocab

{'b b': 0,
 'b b b': 1,
 'b b c': 2,
 'b b e': 3,
 'b b f': 4,
 'b b g': 5,
 'b b h': 6,
 'b b k': 7,
 'b b l': 8,
 'b b n': 9,
 'b b p': 10,
 'b b q': 11,
 'b b r': 12,
 'b b u': 13,
 'b b w': 14,
 'b c': 15,
 'b c b': 16,
 'b c c': 17,
 'b c e': 18,
 'b c f': 19,
 'b c g': 20,
 'b c h': 21,
 'b c k': 22,
 'b c l': 23,
 'b c n': 24,
 'b c p': 25,
 'b c r': 26,
 'b c u': 27,
 'b c v': 28,
 'b c w': 29,
 'b e': 30,
 'b e b': 31,
 'b e c': 32,
 'b e e': 33,
 'b e f': 34,
 'b e g': 35,
 'b e h': 36,
 'b e j': 37,
 'b e k': 38,
 'b e l': 39,
 'b e n': 40,
 'b e p': 41,
 'b e q': 42,
 'b e r': 43,
 'b e u': 44,
 'b e v': 45,
 'b e w': 46,
 'b e x': 47,
 'b e z': 48,
 'b f': 49,
 'b f b': 50,
 'b f c': 51,
 'b f e': 52,
 'b f f': 53,
 'b f g': 54,
 'b f h': 55,
 'b f l': 56,
 'b f n': 57,
 'b f r': 58,
 'b f u': 59,
 'b f w': 60,
 'b g': 61,
 'b g b': 62,
 'b g c': 63,
 'b g e': 64,
 'b g f': 65,
 'b g g': 66,
 'b g h': 67,
 'b g l': 68,
 'b g n': 69,
 'b g p': 70,
 'b g r': 71,
 'b g u': 72

In [50]:
#Get POS tags for tweets and save as a string
tweet_tags = []
for t in tweets:
    tokens = basic_tokenize(preprocess(t))
    tags = nltk.pos_tag(tokens)
    tag_list = [x[1] for x in tags]
    #for i in range(0, len(tokens)):
    tag_str = " ".join(tag_list)
    tweet_tags.append(tag_str)
        #print(tokens[i],tag_list[i])

In [51]:
#We can use the TFIDF vectorizer to get a token matrix for the POS tags
pos_vectorizer = TfidfVectorizer(
    #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    tokenizer=None,
    lowercase=False,
    preprocessor=None,
    ngram_range=(1, 3),
    stop_words=None, #We do better when we keep stopwords
    use_idf=False,
    smooth_idf=False,
    norm=None, #Applies l2 norm smoothing
    decode_error='replace',
    max_features=5000,
    min_df=5,
    max_df=0.501,
    )

In [52]:
#Construct POS TF matrix and get vocab dict
pos = pos_vectorizer.fit_transform(pd.Series(tweet_tags)).toarray()
pos_vocab = {v:i for i, v in enumerate(pos_vectorizer.get_feature_names())}

In [53]:
#Now get other features
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
from textstat.textstat import *

sentiment_analyzer = VS()

def count_twitter_objs(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned.
    +
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, 'URLHERE', parsed_text)
    parsed_text = re.sub(mention_regex, 'MENTIONHERE', parsed_text)
    parsed_text = re.sub(hashtag_regex, 'HASHTAGHERE', parsed_text)
    return(parsed_text.count('URLHERE'),parsed_text.count('MENTIONHERE'),parsed_text.count('HASHTAGHERE'))

def other_features(tweet):
    """This function takes a string and returns a list of features.
    These include Sentiment scores, Text and Readability scores,
    as well as Twitter specific features"""
    ##SENTIMENT
    sentiment = sentiment_analyzer.polarity_scores(tweet)
    
    words = preprocess(tweet) #Get text only
    
    syllables = textstat.syllable_count(words) #count syllables in words
    num_chars = sum(len(w) for w in words) #num chars in words
    num_chars_total = len(tweet)
    num_terms = len(tweet.split())
    num_words = len(words.split())
    avg_syl = round(float((syllables+0.001))/float(num_words+0.001),4)
    num_unique_terms = len(set(words.split()))
    
    ###Modified FK grade, where avg words per sentence is just num words/1
    FKRA = round(float(0.39 * float(num_words)/1.0) + float(11.8 * avg_syl) - 15.59,1)
    ##Modified FRE score, where sentence fixed to 1
    FRE = round(206.835 - 1.015*(float(num_words)/1.0) - (84.6*float(avg_syl)),2)
    
    twitter_objs = count_twitter_objs(tweet) #Count #, @, and http://
    retweet = 0
    if "rt" in words:
        retweet = 1
    features = [FKRA, FRE,syllables, avg_syl, num_chars, num_chars_total, num_terms, num_words,
                num_unique_terms, sentiment['neg'], sentiment['pos'], sentiment['neu'], sentiment['compound'],
                twitter_objs[2], twitter_objs[1],
                twitter_objs[0], retweet]
    #features = pandas.DataFrame(features)
    return features

def get_feature_array(tweets):
    feats=[]
    for t in tweets:
        feats.append(other_features(t))
    return np.array(feats)

In [54]:
other_features_names = ["FKRA", "FRE","num_syllables", "avg_syl_per_word", "num_chars", "num_chars_total", \
                        "num_terms", "num_words", "num_unique_words", "vader neg","vader pos","vader neu", "vader compound", \
                        "num_hashtags", "num_mentions", "num_urls", "is_retweet"]

In [55]:
feats = get_feature_array(tweets)

In [56]:
#Now join them all up
M = np.concatenate([tfidf,pos,feats],axis=1)

In [57]:
M.shape

(24783, 3999)

In [58]:
#Finally get a list of variable names
variables = ['']*len(vocab)
for k,v in vocab.items():
    variables[v] = k

pos_variables = ['']*len(pos_vocab)
for k,v in pos_vocab.items():
    pos_variables[v] = k

feature_names = variables+pos_variables+other_features_names

# Running the model

This model was found using a GridSearch with 5-fold cross validation. Details are in the notebook operational_classifier.

In [59]:
X = pd.DataFrame(M)
y = df['class'].astype(int)

In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC

In [61]:
select = SelectFromModel(LogisticRegression(class_weight='balanced',penalty="l1",C=0.01))
X_ = select.fit_transform(X,y)

C:\Users\obloc_000\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\obloc_000\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [62]:
model = LinearSVC(class_weight='balanced',C=0.01, penalty='l2', loss='squared_hinge',multi_class='ovr').fit(X_, y)

C:\Users\obloc_000\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [63]:
y_preds = model.predict(X_)

In [64]:
report = classification_report( y, y_preds )

In [65]:
print(report)

              precision    recall  f1-score   support

           0       0.49      0.44      0.46      1430
           1       0.92      0.92      0.92     19190
           2       0.75      0.78      0.77      4163

    accuracy                           0.87     24783
   macro avg       0.72      0.71      0.72     24783
weighted avg       0.87      0.87      0.87     24783



# Using information from the model to obtain the matrix X_ generically

This is the most difficult task: We have to take the inputs tweets and transform them into a format that can be used in the model without going through all the same pre-processing steps as above. This can be done as follows.

## Obtaining information about the model

In [66]:
final_features = select.get_support(indices=True) #get indices of features
final_feature_list = [str(feature_names[i]) for i in final_features] #Get list of names corresponding to indices

In [67]:
final_feature_list

['b b',
 'b c',
 'b c h',
 'b e e',
 'b e g',
 'b e l',
 'b e n',
 'b e v',
 'b h n',
 'b l',
 'b l c',
 'b l l',
 'b n',
 'b n b',
 'b n k',
 'b n n',
 'b r',
 'b r e',
 'b r f',
 'b r h',
 'b r n',
 'b r w',
 'b u r',
 'c e',
 'c e h',
 'c e n',
 'c e r',
 'c h e',
 'c h g',
 'c h n',
 'c h r',
 'c k c',
 'c k e',
 'c k f',
 'c k h',
 'c k n',
 'c k u',
 'c l',
 'c l l',
 'c l r',
 'c n',
 'c n f',
 'c n n',
 'c n p',
 'c n r',
 'c p',
 'c r',
 'c r c',
 'c r e',
 'c r h',
 'c r w',
 'c u',
 'c u l',
 'c u n',
 'c u z',
 'c w',
 'e b',
 'e b r',
 'e b u',
 'e c',
 'e c h',
 'e c k',
 'e e',
 'e e b',
 'e e f',
 'e e g',
 'e e h',
 'e e l',
 'e f r',
 'e f u',
 'e g h',
 'e g n',
 'e g u',
 'e h',
 'e h e',
 'e h u',
 'e j',
 'e k e',
 'e k n',
 'e l f',
 'e l r',
 'e l v',
 'e n e',
 'e n k',
 'e n l',
 'e n n',
 'e n u',
 'e p',
 'e p n',
 'e p u',
 'e r',
 'e r c',
 'e r e',
 'e r h',
 'e r l',
 'e r n',
 'e u g',
 'e u p',
 'e w r',
 'e x',
 'e x c',
 'f b r',
 'f e',
 'f e b',
 '

In [35]:
print (final_feature_list)

['b b', 'b c', 'b c h', 'b e e', 'b e g', 'b e l', 'b e n', 'b e v', 'b h n', 'b l', 'b l c', 'b l l', 'b n', 'b n b', 'b n k', 'b n n', 'b r', 'b r e', 'b r f', 'b r h', 'b r n', 'b r w', 'b u r', 'c e', 'c e h', 'c e n', 'c e r', 'c h e', 'c h g', 'c h n', 'c h r', 'c k c', 'c k e', 'c k f', 'c k h', 'c k n', 'c k u', 'c l', 'c l l', 'c l r', 'c n', 'c n f', 'c n n', 'c n p', 'c n r', 'c p', 'c r', 'c r c', 'c r e', 'c r h', 'c r w', 'c u', 'c u l', 'c u n', 'c u z', 'c w', 'e b', 'e b r', 'e b u', 'e c', 'e c h', 'e c k', 'e e', 'e e b', 'e e f', 'e e g', 'e e h', 'e e l', 'e f r', 'e f u', 'e g h', 'e g n', 'e g u', 'e h', 'e h e', 'e h u', 'e j', 'e k e', 'e k n', 'e l f', 'e l r', 'e l v', 'e n e', 'e n k', 'e n l', 'e n n', 'e n u', 'e p', 'e p n', 'e p u', 'e r', 'e r c', 'e r e', 'e r h', 'e r l', 'e r n', 'e u g', 'e u p', 'e w h', 'e w r', 'e x', 'e x c', 'f b r', 'f e', 'f e b', 'f e l', 'f e n', 'f f', 'f g', 'f g e', 'f g g', 'f g h', 'f g r', 'f g u', 'f l h', 'f l k', '

In [76]:
#Getting names for each class of features
ngram_features = final_feature_list[:final_feature_list.index('z z e')+ 1]
pos_features = final_feature_list[final_feature_list.index('z z e')+1:final_feature_list.index('VBZ DT JJ')+1]
oth_features = final_feature_list[final_feature_list.index('FKRA')+1:]


'z z e'

## Generating ngram features

In [77]:
new_vocab = {v:i for i, v in enumerate(ngram_features)}
new_vocab_to_index = {}
for k in ngram_features:
    new_vocab_to_index[k] = vocab[k]

In [78]:
#Get indices of text features
ngram_indices = final_features[:len(ngram_features)]

In [79]:
#TODO: Pickle new vectorizer

In [80]:
new_vectorizer = TfidfVectorizer(
    #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    tokenizer=tokenize,
    preprocessor=preprocess,
    ngram_range=(1, 3),
    stop_words=stopwords, #We do better when we keep stopwords
    use_idf=False,
    smooth_idf=False,
    norm=None, #Applies l2 norm smoothing
    decode_error='replace',
    min_df=1,
    max_df=1.0,
    vocabulary=new_vocab
    )

In [82]:
from sklearn.externals import joblib
joblib.dump(new_vectorizer, '../py3models/final_tfidf.pkl') 

['../py3models/final_tfidf.pkl']

In [83]:
tfidf_ = new_vectorizer.fit_transform(tweets).toarray()

C:\Users\obloc_000\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [84]:
#Verifying that results are the same

In [85]:
tfidf_[1,:]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [86]:
tfidf_[1,:].sum()

23.0

In [87]:
X_[1,:tfidf_.shape[1]]

array([0.        , 1.7828804 , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 3.13038873, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 3.24660201, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 3.19928456, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [88]:
X_[1,:tfidf_.shape[1]].sum()

77.39177416138165

Results are the same if use IDF but the problem is that IDF will be different if we use different data. Instead we have to use the original IDF scores and multiply them by the new matrix.

In [89]:
idf_vals_ = idf_vals[ngram_indices]

In [90]:
idf_vals_.shape

(435,)

In [91]:
#TODO: Pickle idf_vals

joblib.dump(idf_vals_, '../py3models/final_idf.pkl') 

['../py3models/final_idf.pkl']

In [92]:
(tfidf_[1,:]*idf_vals_) == X_[1,:153] #Got same value as final process array!

C:\Users\obloc_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

In [93]:
tfidf_*idf_vals_ == X_[:,:153]

C:\Users\obloc_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

In [95]:
tfidffinal = tfidf_*idf_vals_

## Generating POS features
This is simpler as we do not need to worry about IDF but it will be slower as we have to compute the POS tags for the new data. Here we can simply use the old POS tags.

In [96]:
new_pos = {v:i for i, v in enumerate(pos_features)}

In [97]:
#TODO: Pickle pos vectorizer
#We can use the TFIDF vectorizer to get a token matrix for the POS tags
new_pos_vectorizer = TfidfVectorizer(
    #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    tokenizer=None,
    lowercase=False,
    preprocessor=None,
    ngram_range=(1, 3),
    stop_words=None, #We do better when we keep stopwords
    use_idf=False,
    smooth_idf=False,
    norm=None, #Applies l2 norm smoothing
    decode_error='replace',
    min_df=1,
    max_df=1.0,
    vocabulary=new_pos
    )

In [98]:
joblib.dump(new_pos_vectorizer, '../py3models/final_pos.pkl') 

['../py3models/final_pos.pkl']

In [99]:
pos_ = new_pos_vectorizer.fit_transform(tweet_tags).toarray()

In [100]:
pos_[1,:]

array([0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 2.])

In [101]:
X_[1,153:159]

array([0.        , 0.        , 4.07650155, 0.        , 0.        ,
       0.        ])

In [102]:
pos_[:,:] == X_[:,153:159]

C:\Users\obloc_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

In [103]:
pos_[:,:].sum()

89273.0

In [104]:
X_[:,153:159].sum()

21620.547459814832

## Finally, we can look at the other features

In [106]:
print( other_features_names)

['FKRA', 'FRE', 'num_syllables', 'avg_syl_per_word', 'num_chars', 'num_chars_total', 'num_terms', 'num_words', 'num_unique_words', 'vader neg', 'vader pos', 'vader neu', 'vader compound', 'num_hashtags', 'num_mentions', 'num_urls', 'is_retweet']


In [107]:
print (oth_features)

['FRE', 'num_syllables', 'num_chars', 'num_chars_total', 'num_terms', 'num_words', 'num_unique_words', 'vader neu', 'vader compound', 'num_hashtags', 'num_mentions', 'num_urls']


The functions can be modified to only calculate and return necessary fields.

In [157]:
def other_features_(tweet):
    """This function takes a string and returns a list of features.
    These include Sentiment scores, Text and Readability scores,
    as well as Twitter specific features"""
    ##SENTIMENT
    sentiment = sentiment_analyzer.polarity_scores(tweet)
    
    words = preprocess(tweet) #Get text only
    
    syllables = textstat.syllable_count(words) #count syllables in words
    num_chars = sum(len(w) for w in words) #num chars in words
    num_chars_total = len(tweet)
    num_terms = len(tweet.split())
    num_words = len(words.split())
    avg_syl = round(float((syllables+0.001))/float(num_words+0.001),4)
    num_unique_terms = len(set(words.split()))
    
    ###Modified FK grade, where avg words per sentence is just num words/1
    FKRA = round(float(0.39 * float(num_words)/1.0) + float(11.8 * avg_syl) - 15.59,1)
    ##Modified FRE score, where sentence fixed to 1
    FRE = round(206.835 - 1.015*(float(num_words)/1.0) - (84.6*float(avg_syl)),2)
    
    twitter_objs = count_twitter_objs(tweet) #Count #, @, and http://
    features = [FKRA, FRE, syllables, num_chars, num_chars_total, num_terms, num_words,
                num_unique_terms, sentiment['compound'],
                twitter_objs[2], twitter_objs[1],]
    #features = pandas.DataFrame(features)
    return features

def get_feature_array_(tweets):
    feats=[]
    for t in tweets:
        feats.append(other_features_(t))
    return np.array(feats)

In [166]:
feats_ = get_feature_array_(tweets)

In [163]:
feats_[0,:]

array([   6.5   ,   93.14  ,   26.1   ,  127.    ,  140.    ,   25.    ,
         25.    ,   23.    ,    0.4563,    0.    ,    1.    ])

In [164]:
X_[0,159:]

array([   6.5   ,   93.14  ,   26.1   ,  127.    ,  140.    ,   25.    ,
         25.    ,   23.    ,    0.4563,    0.    ,    1.    ])

In [167]:
feats_[:,:] == X_[:,159:]

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ..., 
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]], dtype=bool)

## Now that we have put it all together using a simplified process we can assess if these new data return the same answers.

In [169]:
M_ = np.concatenate([tfidffinal, pos_, feats_],axis=1)

In [170]:
M_.shape

(24802, 170)

In [176]:
X__ = pd.DataFrame(M_)

In [177]:
y_preds_ = model.predict(X__)

In [178]:
report = classification_report( y, y_preds_ )

In [179]:
print(report)

             precision    recall  f1-score   support

          0       0.46      0.57      0.51      1431
          1       0.97      0.92      0.94     19206
          2       0.82      0.96      0.89      4165

avg / total       0.91      0.90      0.91     24802



OK. So now that we have verified that the results are the same with X_ and X__ we can implement a script that can transform new data in this manner.